In [106]:
import os.path
import pandas as pd
from utils import Wrangling
import warnings
from unidecode import unidecode

warnings.filterwarnings('ignore')

In [107]:
# check if all_data.csv exists, if it doesn't, run function to create it:
if os.path.exists("all_data.csv"):
    data = pd.read_csv("all_data.csv", dtype = str)
else:
    Wrangling.save_tables_from_pdata_cleans()
    data = pd.read_csv("all_data.csv", dtype = str)

data["file_name"] = data["file_name"].astype(str)# file name as string

I had issues with tables that share information from multiple ministerios in one row... check 20220218_135419.pdata_clean and the data_to_fix dataframe generated below. These problematic lines are not currently part of the analyses.

In [108]:
data_to_fix = data[~data["file_name"].str.contains("pdata_clean")] # uncomment this line to check 
# we could potentially keep these:
# data_problem["previsao_de_passageiros"] = data_problem["previsao_de_passageiros"].astype(str)
# data_problem = data_problem[~data_problem["previsao_de_passageiros"].str.contains("pdata_clean")]

Removing problematic lines from the main dataframe for analysis

In [109]:
# remove problematic data:
data["file_name"] = data["file_name"].astype(str)
data = data[data["file_name"].str.contains("pdf")]

# first round of grooming the dataframe - select specific columns, remove accents, extra spaces 
cols = ["autoridades_apoiadas","origem","decolagem_h_local","destino","pouso_h_local","motivo","previsao_de_passageiros","file_name"]
data_clean = Wrangling.clean_flights(data,cols) # to do: combine records removed on this grooming with the data_to_fix

# These are the specific grooming to city names:
data_clean = data_clean.replace("guarulhos", "sao paulo", regex=True)  
data_clean = data_clean.replace("lisboa", "lisbon", regex=True)  
data_clean = data_clean.replace("ascension island", "wide awake", regex=True)  
data_clean = data_clean.replace("port of spain", "port-of-spain", regex=True)  
data_clean = data_clean.replace("madri", "madrid", regex=True)  
data_clean = data_clean.replace("gran canaria island", "gran canaria", regex=True)  
data_clean = data_clean.replace("montevideu", "montevideo", regex=True)  
data_clean = data_clean.replace("^carajas$", "parauapebas", regex=True)  

# select unique - todo: select unique names from "origem" and "destino" and retrieve result into a vector with the number of times thta a given
# city appeared in the flight dataframe
unique_origem = data_clean.groupby('origem').nunique().reset_index()[['origem', 'file_name']]
unique_destino = data_clean.groupby('destino').nunique().reset_index()[['destino', 'file_name']]
# rename cols
unique_origem.columns = ['city','count']
unique_destino.columns = ['city','count']

unique_cities = pd.concat([unique_origem, unique_destino]) # bind origem and destino
del(unique_destino, unique_origem, cols, data_to_fix, data) # clean env

# group again by unique cities and sum count values
unique_cities = unique_cities.groupby('city').sum().reset_index()


Now import the airport names and locations from the whole world using the data published by OpenFlights

In [110]:
airports = pd.read_csv("airports.csv")

cols_names = ["airportid", "name", "city", "country", "iata", "icao", "latitude", "longitude", "altitude", "timezone", "dst", "tz_db", "type", "source"]
airports.columns = cols_names
del(cols_names)

# to lower
airports["city"] = airports["city"].str.lower()
# keep only cols of interest:
airports = airports[["city","country","latitude","longitude"]]

Merge unique cities sourced from FAB flights with airport locations

In [111]:
unique_cities_merge = unique_cities.merge(airports, left_on='city', right_on='city', how="left")
del(unique_cities)
# keep only cities that the location of airports was found in the step above

Find cities that matched with only one airport  

In [112]:
# we will need to perform a set of grooming in the unique_cities_merge. For cities that pass the multiple grooming criteria, 
# we will save those in unique_cities_merge

df = unique_cities_merge.loc[unique_cities_merge['country'].notna()] 
df = df.groupby('city').nunique().reset_index() # count unique 

# separate cities based on the number of combinations from merge
df = df[df['latitude'] == 1]
# update country, lat and long columns
df = df[["city"]].merge(airports, left_on='city', right_on='city', how="left")

# copy to unique_cities_clean
unique_cities_clean = df
del(df)


Start with fixing
1- check cities that did not have any matches with the airports from OpenFlights

In [113]:
# save the cities that did not have any matches in the merge above
df = unique_cities_merge.loc[unique_cities_merge['country'].isna()] 

# I reviewed all cities that were recorded more than 10 times
df = df[df['count'] > 10]
df = df[["city"]]

## to do: combine again with airports and then with brazilian airports dataframe
df = df.merge(airports, left_on='city', right_on='city', how="left")
# if cities appear in more than one country take the first option
df = df.groupby(["city"]).head(1)

# keep working with cities that still don't have lat lon and save those that were fixed in this step to the clean df:
unique_cities_clean = pd.concat([unique_cities_clean, df.loc[df['latitude'].notna()]])
df = df.loc[df['latitude'].isna()] 

In [114]:
br_cities = pd.read_csv("brazilian_cities.csv")
br_cities["nome"] = br_cities["nome"].str.lower()
br_cities["nome"] = br_cities["nome"].apply(unidecode) # replace letters with accents with the letter without accent.

# fix names:

# to do: only merge records without lat lon at this point and then group by and select one
df = df[["city", "country"]].merge(br_cities[["nome", "latitude", "longitude"]], left_on='city', right_on='nome', how="left")[["city", "country", "latitude", "longitude"]]

# country is Brazil
df["country"] = "Brazil"
# if cities appear in more than one state take the first option
df = df.groupby(["city"]).head(1)

# only two cities were not found: praia and ilha do sal, these will be removed from the analysis
unique_cities_clean = pd.concat([unique_cities_clean, df.loc[df['latitude'].notna()]])
del(df)

3 - find cities that matched with more than one airport and select just one combination (in case airport appeared more than 10 times in the data dataframe)
    - if one of the airports is in Brazil, keep it and remove the others (see vitoria)

In [126]:
df = unique_cities_merge.loc[unique_cities_merge['country'].notna()] 
df = df.groupby('city').nunique().reset_index() # count unique 

# separate cities based on the number of combinations from merge
df = df[df['latitude'] > 1]

# # update country, lat and long columns
df = df[["city"]].merge(airports, left_on='city', right_on='city', how="left")

# # .. and where one of them is in Brazil, choose that one.
df_br = df[df['country'].isin(["Brazil"])]
df_br = df_br.groupby('city').head(1)

# # from df, remove cities that were resolved above
sel = list(df_br['city']) # select cities that matched with more than one airport
df = df[~df['city'].isin(sel)]
df = df.groupby('city').head(1)

# save to clean df:
unique_cities_clean = pd.concat([unique_cities_clean, df, df_br])
del(df, df_br, sel)


 - if more than one airport is in a city in Brazil (see sao paulo), take the first airport location
 - if none of the airports is in brazil (use column from merge above to remove cities in brazil), group by country + city, select combination with country > airports, merge again with locations and select head

 https://stackoverflow.com/questions/53842287/select-rows-with-highest-value-from-groupby